In [14]:
# Importing necessary libraries
import numpy as np
import random
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop




# Reading the text file into a string
with open('./sample.txt', 'r') as file:
    text = file.read()

# A preview of the text file
print(text)

# Storing all the unique characters present in the text
vocabulary = sorted(list(set(text)))

# Creating dictionaries to map each character to an index
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

# Adjust max_length according to your needs
max_length = 100
steps = 5
sentences = []
next_chars = []

# Generating subsequences of length max_length
for i in range(0, len(text) - max_length, steps):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])

# Hot encoding each character into a boolean vector
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype=np.bool_)
y = np.zeros((len(sentences), len(vocabulary)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

# Building the LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Function to sample an index from a probability array
def sample_index(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Load pre-trained weights
filepath = "weights.keras"
model.load_weights(filepath)

# Generating new text conditioned on the provided context
def generate_text(length, diversity, input):
    # Initialize with the given context
    generated = ""
    sentence = input
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')

    # Predict next characters based on the context
    for i in range(length):
        x_pred = np.zeros((1, max_length, len(vocabulary)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample_index(preds, diversity)
        next_char = indices_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

# Generate 200 new characters conditioned on the context
context = "The United States used another comeback yesterday to take"

generated_text = generate_text(200, 0.2, context)
print(generated_text)



@@3000041  Basketball United States Wins Bronze Medal : The United States used another comeback yesterday to take the bronze medal at the world championships in Buenos Aires , beating Puerto Rico by 107-105 in overtime . Kenny Anderson scored 34 points and made two free throws with one second left to send the game into overtime after Puerto Rico led by 8 points with 1 minute 34 seconds to play in the second half . Yugoslavia , which beat the United States in the semifinals , will face the Soviet Union for the gold medal today . This was the fourth straight international tournament in which the United States had failed to win a gold . Any medal for the United States looked out of the question with 1:34 left in regulation when Raymond Gausse 's 3-pointer gave Puerto Rico a 96-88 lead . Then Anderson hit a jumper with 1:21 to play to get the United States within 6 , and Puerto Rico opted for possession rather than foul shots four times in the final 1:19 . After a backcourt violation , An

----- Generating with seed: "The United States used another comeback yesterday to take"
The United States used another comeback yesterday to takea'iraa   iarraiaiiaaa000a a    iaga  br i raaiaaiinai ra rriiii aiaiiaai   riaaiaa  ynwr riaigiaai  nn i  ai     trabrriaa'aarannaaaaaanararariin ar agiaaa   t  b  aria  a  0ar raaia0rriiai   a  0iacr


In [13]:
print(generate_text(500, 0.2))

TypeError: generate_text() missing 1 required positional argument: 'input'